# Mixed Data Distances 
Rowan Data Mining 1, Homework 4  
Due: 2018.07.30  
Ron Neely


In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import DistanceMetric as dm
import sys
print(sys.version)
print(pd.__version__)
print(np.__version__)

3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
0.22.0
1.14.2


## read the data

In [2]:
df_raw = pd.read_csv("dif_sim.csv")
print(df_raw.dtypes)
df_raw

name          object
gender        object
fav_color     object
blood_type    object
gen_health    object
test1          int64
cough         object
high_bp       object
dtype: object


,name,gender,fav_color,blood_type,gen_health,test1,cough,high_bp
0,Susan,F,Blue,O-,excellent,75,N,N
1,Jim,M,Red,O+,good,65,N,N
2,Joe,M,Red,AB-,fair,64,N,Y
3,Jane,F,Green,A+,poor,83,Y,Y
4,Sam,M,Blue,A-,good,71,N,N
5,Michelle,F,Blue,O-,good,90,N,N


## clean and normalize the data 
Save a lowercase dataframe for the challenge question at the end.

In [3]:
df_lower = df_raw.applymap(lambda s:s.lower() if type(s) == str else s)
df_lower.set_index('name', inplace=True, drop=True)
df = df_lower.copy()
df

,gender,fav_color,blood_type,gen_health,test1,cough,high_bp
name,,,,,,,
susan,f,blue,o-,excellent,75,n,n
jim,m,red,o+,good,65,n,n
joe,m,red,ab-,fair,64,n,y
jane,f,green,a+,poor,83,y,y
sam,m,blue,a-,good,71,n,n
michelle,f,blue,o-,good,90,n,n


In [4]:
def color1(s):
    if 'red' == s: 
        return 0.0
    elif 'green' == s:
        return 0.5
    else:
        return 1.0

def blood_type1(s):
    if 'o-' == s: 
        return 0.0
    elif 'o+' == s:
        return 0.25
    elif 'a-' == s:
        return 0.5
    elif 'a+' == s:
        return 0.75
    else:
        return 1.0

def gen_health1(s):
    if 'poor' == s: 
        return 0.0
    elif 'fair' == s:
        return 0.333
    elif 'good' == s:
        return 0.666
    else:
        return 1.0
    
df.gender = df.gender.map(lambda s : int(1) if s == "m" else int(0))
df.fav_color = df.fav_color.map(color1)
df.blood_type = df.blood_type.map(blood_type1)
df.gen_health = df.gen_health.map(gen_health1)
df.test1 = df.test1.map(lambda v: (float(v)-df.test1.min())/(df.test1.max() - df.test1.min()))
df.cough = df.cough.map(lambda s : int(1) if s == "y" else int(0))
df.high_bp = df.high_bp.map(lambda s : int(1) if s == "y" else int(0))
print(df.dtypes)
df

gender          int64
fav_color     float64
blood_type    float64
gen_health    float64
test1         float64
cough           int64
high_bp         int64
dtype: object


,gender,fav_color,blood_type,gen_health,test1,cough,high_bp
name,,,,,,,
susan,0,1.0,0.00,1.000,0.423077,0,0
jim,1,0.0,0.25,0.666,0.038462,0,0
joe,1,0.0,1.00,0.333,0.000000,0,1
jane,0,0.5,0.75,0.000,0.730769,1,1
sam,1,1.0,0.50,0.666,0.269231,0,0
michelle,0,1.0,0.00,0.666,1.000000,0,0


## Compute 5 difference matrcies (for each of the 5 data types):

- categorical ['fav_color', 'blood_type']
- symmetric binary ['gender', 'high_bp'] 
- assymmetric binary ['cough']
- ordinal ['gen_health']
- numeric ['test1']

Define a function to compute a difference matrix from certain columns of a Dataframe with a difference kernels.

In [5]:
def diff_matrix(dataframe, columns, diff_kernel):
    ar = np.zeros((len(dataframe), len(dataframe)))
    i = 0
    for i_row in dataframe[columns].iterrows():
        j = 0
        for j_row in dataframe[columns].iterrows():
            ar[i,j] = diff_kernel(i_row, j_row)
            j+=1
        i+=1    
    ddf = pd.DataFrame(data=ar, columns=df.index, index=df.index)
    return ddf

### dfcd: difference - categorical ['fav_color', 'blood_type']
difference function is *1- (matches / variables)*

In [6]:
def diff_cat(A, B):
    cnt_vars = len(A[1])
    cnt_match = 0
    for _, row in enumerate(zip(A[1],B[1])):
        a, b = row
        if a == b:
            cnt_match += 1
    diff = 1 - (cnt_match / cnt_vars)
    return diff

dfcd = diff_matrix(df, ['fav_color', 'blood_type'], diff_cat)
dfcd

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.0,1.0,1.0,1.0,0.5,0.0
jim,1.0,0.0,0.5,1.0,1.0,1.0
joe,1.0,0.5,0.0,1.0,1.0,1.0
jane,1.0,1.0,1.0,0.0,1.0,1.0
sam,0.5,1.0,1.0,1.0,0.0,0.5
michelle,0.0,1.0,1.0,1.0,0.5,0.0


### dfsd: difference - symmetric binary ['gender', 'high_bp'] 

Given:

| A | B | Sum |
|---|---|---|
| 1 | 1 | q |
| 0 | 1 | r |
| 1 | 0 | s |
| 0 | 0 | t |

difference function is $$ \frac{r+s}{q+r+s+t}$$

In [7]:
def diff_bin_sym(A, B):
    q = r = s = t = 0
    for _, row in enumerate(zip(A[1],B[1])):
        a, b = row
        if a:
            if b:
                q += 1
            else:
                s += 1
        else:
            if b:
                r += 1
            else:
                t += 1
    num = r + s
    den = q + r + s + t
    if den:
        diff = num / den
    else:
        diff = num
    return diff

dfsd = diff_matrix(df, ['gender', 'high_bp'], diff_bin_sym)
dfsd

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.0,0.5,1.0,0.5,0.5,0.0
jim,0.5,0.0,0.5,1.0,0.0,0.5
joe,1.0,0.5,0.0,0.5,0.5,1.0
jane,0.5,1.0,0.5,0.0,1.0,0.5
sam,0.5,0.0,0.5,1.0,0.0,0.5
michelle,0.0,0.5,1.0,0.5,0.5,0.0


### dfad: difference - assymmetric binary ['cough']
difference function is *(r+s)/(q+r+s)*

In [8]:
def diff_bin_asym(A, B):
    q = r = s = t = 0
    for _, row in enumerate(zip(A[1],B[1])):
        a, b = row
        if a:
            if b:
                q += 1
            else:
                s += 1
        else:
            if b:
                r += 1
            else:
                t += 1
    num = r + s
    den = q + r + s
    if den:
        diff = num / den
    else:
        diff = num
    return diff

dfad = diff_matrix(df, ['cough'], diff_bin_asym)
dfad

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.0,0.0,0.0,1.0,0.0,0.0
jim,0.0,0.0,0.0,1.0,0.0,0.0
joe,0.0,0.0,0.0,1.0,0.0,0.0
jane,1.0,1.0,1.0,0.0,1.0,1.0
sam,0.0,0.0,0.0,1.0,0.0,0.0
michelle,0.0,0.0,0.0,1.0,0.0,0.0


### dfod: difference - ordinal ['gen_health']
difference function is normalized weighted euclidean distance (let weighting=1)

$$
{\displaystyle {\begin{aligned}d(i ,j )=\frac{\sqrt{ w_{1}(x_{1i}-x_{1i})^{2} + w_{2}(x_{2i}-x_{2i})^{2} + ... + w_{n}(x_{ni}-x_{ni})^{2} }}{ \sqrt{w_{1} + ... + w_{n} } }\end{aligned}}}
$$

In [9]:
def diff_num(A, B, W=None):
    if not W:
        W = pd.Series(np.ones(len(A)))
    sum_wdiff2 = 0
    sum_w = 0
    for _, row in enumerate(zip(A[1], B[1], W.values)):
        a, b, w = row
        sum_wdiff2 += w * math.pow(a - b, 2)
        sum_w += w
    diff = math.sqrt(sum_wdiff2) / math.sqrt(sum_w)
    return diff

dfod = diff_matrix(df, ['gen_health'], diff_num)
dfod

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.000,0.334,0.667,1.000,0.334,0.334
jim,0.334,0.000,0.333,0.666,0.000,0.000
joe,0.667,0.333,0.000,0.333,0.333,0.333
jane,1.000,0.666,0.333,0.000,0.666,0.666
sam,0.334,0.000,0.333,0.666,0.000,0.000
michelle,0.334,0.000,0.333,0.666,0.000,0.000


### dfnd: difference - numeric ['test1']
difference function is normalized weighted euclidean distance (let weighting=1)

In [10]:
dfnd = diff_matrix(df, ['test1'], diff_num)
dfnd

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.000000,0.384615,0.423077,0.307692,0.153846,0.576923
jim,0.384615,0.000000,0.038462,0.692308,0.230769,0.961538
joe,0.423077,0.038462,0.000000,0.730769,0.269231,1.000000
jane,0.307692,0.692308,0.730769,0.000000,0.461538,0.269231
sam,0.153846,0.230769,0.269231,0.461538,0.000000,0.730769
michelle,0.576923,0.961538,1.000000,0.269231,0.730769,0.000000


## Convert all to Similarity Matrices
similarity matrices are elementwise 1.0 - difference 

### similarity - categorical ['fav_color', 'blood_type']
1.0 - dfcd

In [11]:
ones = np.ones((len(df), len(df)))
dfcs = ones - dfcd
dfcs

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.0,0.0,0.0,0.0,0.5,1.0
jim,0.0,1.0,0.5,0.0,0.0,0.0
joe,0.0,0.5,1.0,0.0,0.0,0.0
jane,0.0,0.0,0.0,1.0,0.0,0.0
sam,0.5,0.0,0.0,0.0,1.0,0.5
michelle,1.0,0.0,0.0,0.0,0.5,1.0


### similarity - symmetric binary ['gender', 'high_bp'] 
1.0 - dfsd

In [12]:
dfss = ones - dfsd
dfss

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.0,0.5,0.0,0.5,0.5,1.0
jim,0.5,1.0,0.5,0.0,1.0,0.5
joe,0.0,0.5,1.0,0.5,0.5,0.0
jane,0.5,0.0,0.5,1.0,0.0,0.5
sam,0.5,1.0,0.5,0.0,1.0,0.5
michelle,1.0,0.5,0.0,0.5,0.5,1.0


### similarity - assymmetric binary ['cough']
1.0 - dfad

In [13]:
dfas = ones - dfad

### similarity - ordinal ['gen_health']
1.0 - dfod

In [14]:
dfos = ones - dfod

### similarity - numeric ['test1']
1.0 - dfnd

In [15]:
dfns = ones - dfnd
dfns

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.000000,0.615385,0.576923,0.692308,0.846154,0.423077
jim,0.615385,1.000000,0.961538,0.307692,0.769231,0.038462
joe,0.576923,0.961538,1.000000,0.269231,0.730769,0.000000
jane,0.692308,0.307692,0.269231,1.000000,0.538462,0.730769
sam,0.846154,0.769231,0.730769,0.538462,1.000000,0.269231
michelle,0.423077,0.038462,0.000000,0.730769,0.269231,1.000000


## Add 5 together (to create overall similarity matrix)
Don't forget to give twice the weight for the nominal and asymmetric binary matrices.
Divide each entry by 7 to get a single 6x6 similarity matrix with entries between 0 and 1.
Note: if you don’t get a symmetric matrix with 1’s on the diagonal then you did something wrong!



In [16]:
dfs = (2 * dfcs + dfss + 2 * dfas + dfos + dfns) / 7
dfs

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.000000,0.540198,0.415703,0.170330,0.716022,0.869868
jim,0.540198,1.000000,0.732648,0.091670,0.681319,0.505495
joe,0.415703,0.732648,1.000000,0.205176,0.556824,0.381000
jane,0.170330,0.091670,0.205176,1.000000,0.124637,0.223538
sam,0.716022,0.681319,0.556824,0.124637,1.000000,0.681319
michelle,0.869868,0.505495,0.381000,0.223538,0.681319,1.000000


## Ask and answer a challenge question of your choosing for the 10th point.
Using various python libraries what is the overall cosine similarity matrix between all the records utitlizing dummy variables for the categorical values?
http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html And, how different is it from the multicategorical simularity matrix created above?

### Normalize data with dummies
Start with df_lower.  Use dummy variables where applicable.

First, normalize gen_health as an ordinal, and test1.

In [17]:
df = df_lower.copy()
df.gen_health = df.gen_health.map(gen_health1)
df.test1 = (df.test1 -df.test1.min())/(df.test1.max()-df.test1.min())
df

,gender,fav_color,blood_type,gen_health,test1,cough,high_bp
name,,,,,,,
susan,f,blue,o-,1.000,0.423077,n,n
jim,m,red,o+,0.666,0.038462,n,n
joe,m,red,ab-,0.333,0.000000,n,y
jane,f,green,a+,0.000,0.730769,y,y
sam,m,blue,a-,0.666,0.269231,n,n
michelle,f,blue,o-,0.666,1.000000,n,n


In [18]:
df = pd.get_dummies(df)
print(df.dtypes)

gen_health         float64
test1              float64
gender_f             uint8
gender_m             uint8
fav_color_blue       uint8
fav_color_green      uint8
fav_color_red        uint8
blood_type_a+        uint8
blood_type_a-        uint8
blood_type_ab-       uint8
blood_type_o+        uint8
blood_type_o-        uint8
cough_n              uint8
cough_y              uint8
high_bp_n            uint8
high_bp_y            uint8
dtype: object


Several of the above dummy columns are redundant.  Drop them.

In [19]:
df=df.drop(['gender_f','fav_color_blue','blood_type_ab-','cough_n','high_bp_n'],
        axis=1)
df

,gen_health,test1,gender_m,fav_color_green,fav_color_red,blood_type_a+,blood_type_a-,blood_type_o+,blood_type_o-,cough_y,high_bp_y
name,,,,,,,,,,,
susan,1.000,0.423077,0,0,0,0,0,0,1,0,0
jim,0.666,0.038462,1,0,1,0,0,1,0,0,0
joe,0.333,0.000000,1,0,1,0,0,0,0,0,1
jane,0.000,0.730769,0,1,0,1,0,0,0,1,1
sam,0.666,0.269231,1,0,0,0,1,0,0,0,0
michelle,0.666,1.000000,0,0,0,0,0,0,1,0,0


In [20]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, cosine

res = pdist(df, 'cosine')
distance = pd.DataFrame(squareform(res), index=df.index, columns= df.index)
distance

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.000000,0.750981,0.872099,0.901637,0.666915,0.094653
jim,0.750981,0.000000,0.321325,0.992888,0.506165,0.833867
joe,0.872099,0.321325,0.000000,0.733734,0.563292,0.919561
jane,0.901637,0.992888,0.733734,0.000000,0.941749,0.780453
sam,0.666915,0.506165,0.563292,0.941749,0.000000,0.712532
michelle,0.094653,0.833867,0.919561,0.780453,0.712532,0.000000


In [21]:
similar = ones - distance
similar

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.000000,0.249019,0.127901,0.098363,0.333085,0.905347
jim,0.249019,1.000000,0.678675,0.007112,0.493835,0.166133
joe,0.127901,0.678675,1.000000,0.266266,0.436708,0.080439
jane,0.098363,0.007112,0.266266,1.000000,0.058251,0.219547
sam,0.333085,0.493835,0.436708,0.058251,1.000000,0.287468
michelle,0.905347,0.166133,0.080439,0.219547,0.287468,1.000000


Recall dfs is our calculated overal simlarity matrix.

In [22]:
dfs

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,1.000000,0.540198,0.415703,0.170330,0.716022,0.869868
jim,0.540198,1.000000,0.732648,0.091670,0.681319,0.505495
joe,0.415703,0.732648,1.000000,0.205176,0.556824,0.381000
jane,0.170330,0.091670,0.205176,1.000000,0.124637,0.223538
sam,0.716022,0.681319,0.556824,0.124637,1.000000,0.681319
michelle,0.869868,0.505495,0.381000,0.223538,0.681319,1.000000


They are roughly close.  Let's see how close.

In [23]:
dif_sim = similar - dfs
dif_sim = dif_sim.applymap(abs)
dif_sim

name,susan,jim,joe,jane,sam,michelle
name,,,,,,
susan,0.000000,0.291179,0.287802,0.071967,0.382937,0.035479
jim,0.291179,0.000000,0.053974,0.084559,0.187483,0.339362
joe,0.287802,0.053974,0.000000,0.061090,0.120116,0.300561
jane,0.071967,0.084559,0.061090,0.000000,0.066386,0.003992
sam,0.382937,0.187483,0.120116,0.066386,0.000000,0.393851
michelle,0.035479,0.339362,0.300561,0.003992,0.393851,0.000000


In [24]:
sorted(set(dif_sim.values.flatten().tolist()), reverse=True)[0:10]


[0.393850615621799,
 0.382937046822455,
 0.3393619081943654,
 0.30056129746967425,
 0.29117861798798894,
 0.28780229364755433,
 0.18748339472575526,
 0.12011615520364749,
 0.08455871567349342,
 0.07196714429612544]

Overall cosine similarity finds sam to be more different to michelle and susan than our method calculated.
It also finds michelle and jim more different.
